In [135]:
# Import necessary libraries
import cv2 as cv
import numpy as np
import insightface
import pickle
import os
import mediapipe as mp
from insightface.app import FaceAnalysis
from numpy.linalg import norm

In [ ]:
# Load the model
detector = FaceAnalysis(name="buffalo_l",  providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640), det_thresh=0.5)

c:\Users\User\anaconda3\envs\environment\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\User/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

# Data Acquisition

In [ ]:
# Define image repository path
img_repo_path = "captured_repo/"

# Load camera
video = cv.VideoCapture(0)

while True:
    ret, frame = video.read()
    if not ret:
        break

    original_frame = frame.copy()

    faces = detector.get(frame)

    for face in faces:
        bbox = face.bbox.astype(int)
        cv.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)

    cv.imshow("Face Detection", frame)

    key = cv.waitKey(1) & 0xFF
    if key == ord('s'):
        if len(faces) != 0:
            print("Image Captured.")
            cv.imwrite(img_path, original_frame) 
            break
    elif key == ord('q'):
        break

video.release()
cv.destroyAllWindows()


# Image Processing

need to adjust again for the preprocessing step

only need to convert the bgr to rgb, adjust the brightness automatically

In [138]:
img = cv.imread(img_path)
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

# Contrast enhancement
lab = cv.cvtColor(img_rgb, cv.COLOR_RGB2LAB)
l, a, b = cv.split(lab)
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
l_clahe = clahe.apply(l)
lab_clahe = cv.merge((l_clahe, a, b))
img_rgb = cv.cvtColor(lab_clahe, cv.COLOR_LAB2RGB)

# Bilateral filter
img_rgb = cv.bilateralFilter(img_rgb, d=9, sigmaColor=75, sigmaSpace=75)

# Face Detection

In [139]:
faces = detector.get(img_rgb)
h, w = img.shape[:2]
frame_center = np.array([w / 2, h / 2])

main_face = min(faces, key=lambda f: np.linalg.norm(
    np.array([(f.bbox[0] + f.bbox[2]) / 2, (f.bbox[1] + f.bbox[3]) / 2]) - frame_center
))

bbox = main_face.bbox.astype(int)
blurred_img = cv.GaussianBlur(img_rgb, (51,51),0)
mask = np.zeros((h, w), dtype=np.uint8)
cv.rectangle(mask, (bbox[0], bbox[1]), (bbox[2], bbox[3]), 255, -1)
mask_3ch = cv.merge([mask, mask, mask])
result = np.where(mask_3ch == 255, img, blurred_img)

cv.imshow("Focused Main Face", result)
cv.waitKey(0)
cv.destroyAllWindows()

# Feature Extraction

In [140]:
embedding = main_face.normed_embedding

# Face Recognition

In [141]:
with open("processed_data/face_embeddings.pkl", "rb") as file:
    embeddings_data = pickle.load(file)

In [142]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def find_top_k_matches(query_embedding, embeddings_data, k=5):
    similarities = []
    for item in embeddings_data:
        sim = cosine_similarity(query_embedding, item["embedding"])
        similarities.append({
            "id": item["id"],
            "image_name": item["image_name"],
            "similarity": sim,
            "landmarks": item.get("landmarks", []) 
        })
    return sorted(similarities, key=lambda x: x["similarity"], reverse=True)[:k]

In [143]:
top_matches = find_top_k_matches(embedding, embeddings_data, k=2)
if top_matches:
        print("Top matches:")
        for match in top_matches:
            print(f"ID: {match['id']}, Similarity: {match['similarity']:.3f}")

Top matches:
ID: 2408866, Similarity: 0.484
ID: 2408863, Similarity: 0.218


# Post Processing

In [144]:
# Get the top 1 match
top1 = top_matches[0]
print(f"Top match: {top1}")

# Load the matched image
matched_img = cv.imread(img_path)
matched_img_rgb = cv.cvtColor(matched_img, cv.COLOR_BGR2RGB)

# Detect faces in the matched image
faces = detector.get(matched_img_rgb)

if faces:
    for face in faces:
        # Get bounding box
        x1, y1, x2, y2 = face.bbox.astype(int)
        
        # Draw rectangle
        cv.rectangle(matched_img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw label above the rectangle
        label = f"Top 1: {top1['image_name']} ({top1['similarity']:.2f})"
        cv.putText(matched_img_rgb, label, (x1, y1 - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
else:
    print("No face detected in matched image.")

# Show the image
cv.imshow("Top 1 Match", cv.cvtColor(matched_img_rgb, cv.COLOR_RGB2BGR))
cv.waitKey(0)
cv.destroyAllWindows()

Top match: {'id': '2408866', 'image_name': '2408866.jpg', 'similarity': np.float32(0.4835062), 'landmarks': []}
